In [1]:
from langchain.document_loaders import YoutubeLoader
from langchain.llms import OpenAI
from langchain.chains.summarize import load_summarize_chain

In [2]:
OPENAI_API_KEY = '...'

> ! pip install youtube-transcript-api

> ! pip install pytube

### 1. Simple Videos

In [20]:
loader = YoutubeLoader.from_youtube_url("https://www.youtube.com/watch?v=mWIBTkwUJf8", add_video_info=True)

In [21]:
result = loader.load()

In [22]:
print (type(result))
print (f"Found video from {result[0].metadata['author']} that is {result[0].metadata['length']} seconds long")
print ("")
print (result)

<class 'list'>
Found video from Saturday Night Live that is 572 seconds long

[Document(page_content='-Is it alright,\nsitting back like this? Too casual, too...? The first time I auditioned,\nI remember walking underneath all these pipes\nthat you\'ve seen, you know, in sketches, whenever\nthey do backstage things. I go, "This is really surreal." Like I was here once before. My Uncle George,\nwhen he played Norm on "Cheers" and he hosted the finale\nof 1991, so I got to come watch\nthat show and so this was the second time\nI got to be, you know, in the space\nand here it was, you know, 12 years later,\na dozen years later. And then turn and then\nthere\'s the stage and the brass are sitting there\nat a table and you\'re just doing it,\nbut to no laughs. You\'re like, "Ah, [bleep]\nAlright, okay." [ Laughs ] ♪♪ I guess it was like August 2003, I was doing a Second City show\nin Las Vegas at the Flamingo. We were performing\nfor no one. We were doing shows\nfor like eight people. I rem

In [23]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

In [24]:
chain = load_summarize_chain(llm, chain_type="stuff", verbose=False)
chain.run(result)

' Jason Sudeikis recounts his experience of auditioning for and eventually joining the cast of Saturday Night Live. He talks about his journey from being a performer to a writer and then to a cast member. He also talks about his experience of leaving the show after 10 years and how it changed his life.'

### 2. Long Videos

In [29]:
loader = YoutubeLoader.from_youtube_url("https://www.youtube.com/watch?v=yIut0M9dq24", add_video_info=True)

In [30]:
result = loader.load()

In [31]:
chain = load_summarize_chain(llm, chain_type="stuff", verbose=False)
chain.run(result)

InvalidRequestError: This model's maximum context length is 4097 tokens, however you requested 5635 tokens (5379 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.

Problem, your transcript/document is too long

In [32]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [33]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)

In [34]:
texts = text_splitter.split_documents(result)

In [35]:
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=True)
chain.run(texts[:4])



> Entering new MapReduceDocumentsChain chain...
Prompt after formatting:
Write a concise summary of the following:


"- I'm from Staten Island.
I'm sorry, everybody. If you don't know what Staten island is, it's like New York's abortion that lived. So I would listen to "Gangster's Paradise" and it would make me, you
know, like maybe I can get out of Staten Island. - Guy from New York! It's Pete Davidson. (audience laughing) - Yay! I just dropped out of
college. Anybody else? (audience cheering) Yeah. I dropped out cause I was dorming
and I didn't like dorming. That's it. (audience laughing) I didn't speak to any of my roommates. And I think it was cause
like the first morning we all woke up with a
boner and we were like, this isn't for any of us. (audience laughing) One time we ran out of toilet paper and we just didn't get any. (audience laughing) But everybody took a shit every day and nobody knew what they used. It was like this don't ask,
don't tell policy that we had. (audience 

' Pete Davidson is a comedian from Staten Island who talks about his experiences in an all-boys Catholic school, his pot smoking, and his roommates. He also talks about playing Call of Duty, his friends being black, and his hometown. He shares a humorous story about his mother barging into his room late at night and giving him a 40-pack of condoms. He jokes about the situation and other people in the audience.'

### 3. Multiple Videos

In [36]:
youtube_url_list = ["https://www.youtube.com/watch?v=mWIBTkwUJf8", "https://www.youtube.com/watch?v=yIut0M9dq24"]

texts = []

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)

for url in youtube_url_list:
    loader = YoutubeLoader.from_youtube_url(url, add_video_info=True)
    result = loader.load()
    
    texts.extend(text_splitter.split_documents(result))

In [37]:
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=False)
chain.run(texts)

' This is a story about a 16-year-old kid who ended up in the cast of Saturday Night Live. He was asked to write something he thought he could score on, and ended up staying for three episodes. He reflects on his experiences auditioning, writing, and performing on the show, as well as his time as a DJ at a strip club. He also talks about his friends, his hometown of Staten Island, and his favorite rapper, Coolio. He encourages viewers to stick around for the upcoming show featuring Brandi Carlile.'